In [1]:
import torch
import torchvision
import torchvision.transforms as transforms
import numpy as np
import torch.nn as nn
import torch.nn.functional as F
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [2]:
#  Download and load CIFAR-10 dataset
transform = transforms.Compose([transforms.Grayscale(num_output_channels=1), transforms.ToTensor()])
trainset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=64, shuffle=True)

testset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=64, shuffle=False)

def compute_svd_singular_values(image_batch):
    singular_values = []
    for image in image_batch:
        U, S, V = torch.svd(image.squeeze())
        singular_values.append(S)
    return torch.stack(singular_values)

# Compute singular values for train and test sets
train_singular_values = [compute_svd_singular_values(batch[0]).to(device) for batch in trainloader]
test_singular_values = [compute_svd_singular_values(batch[0]).to(device) for batch in testloader]

100.0%


Extracting ./data/cifar-10-python.tar.gz to ./data
Files already downloaded and verified


In [14]:
len(train_singular_values[0])

64

### Step 2: Neural Network Training

#### A. Define the Neural Network Model

In [4]:
class SVDPredictor(nn.Module):
    def __init__(self):
        super(SVDPredictor, self).__init__()
        self.fc1 = nn.Linear(32*32, 512)
        self.fc2 = nn.Linear(512, 256)
        self.fc3 = nn.Linear(256, 32)  # Predicting 32 singular values

    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = F.dropout(x, p=0.5)
        x = F.relu(self.fc2(x))
        x = F.dropout(x, p=0.5)
        return self.fc3(x)

# move the model to gpu
model = SVDPredictor().to(device)

#### B. Define Loss Function and Optimizer

In [14]:
import torch.optim as optim

criterion = nn.L1Loss()
optimizer = optim.Adam(model.parameters(), lr=0.00001)


#### C. Training the Model

In [15]:
# Training loop
num_epochs = 50  # Set the number of epochs
print_every_n_batches = 100  # Print log after every 100 batches

for epoch in range(num_epochs):
    running_loss = 0.0
    for i, (images, _) in enumerate(trainloader):
        images = images.view(images.shape[0], -1).to(device)
        labels = train_singular_values[i]  # Precomputed singular values

        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

        # Log the running loss
        if (i + 1) % print_every_n_batches == 0:
            print(f'Epoch [{epoch + 1}/{num_epochs}], Step [{i + 1}/{len(trainloader)}], Loss: {running_loss / print_every_n_batches:.4f}')
            running_loss = 0.0

print('Finished Training')



Epoch [1/50], Step [100/782], Loss: 0.2069
Epoch [1/50], Step [200/782], Loss: 0.2057
Epoch [1/50], Step [300/782], Loss: 0.2046
Epoch [1/50], Step [400/782], Loss: 0.2032
Epoch [1/50], Step [500/782], Loss: 0.2048
Epoch [1/50], Step [600/782], Loss: 0.2059
Epoch [1/50], Step [700/782], Loss: 0.2057
Epoch [2/50], Step [100/782], Loss: 0.2070
Epoch [2/50], Step [200/782], Loss: 0.2056
Epoch [2/50], Step [300/782], Loss: 0.2053
Epoch [2/50], Step [400/782], Loss: 0.2036
Epoch [2/50], Step [500/782], Loss: 0.2047
Epoch [2/50], Step [600/782], Loss: 0.2058
Epoch [2/50], Step [700/782], Loss: 0.2051
Epoch [3/50], Step [100/782], Loss: 0.2066
Epoch [3/50], Step [200/782], Loss: 0.2056
Epoch [3/50], Step [300/782], Loss: 0.2056
Epoch [3/50], Step [400/782], Loss: 0.2038
Epoch [3/50], Step [500/782], Loss: 0.2045
Epoch [3/50], Step [600/782], Loss: 0.2059
Epoch [3/50], Step [700/782], Loss: 0.2051
Epoch [4/50], Step [100/782], Loss: 0.2065
Epoch [4/50], Step [200/782], Loss: 0.2058
Epoch [4/50

### Step 3: Evaluation


##### Implementing NMSE and MAE Computation

In [16]:
def compute_nmse(outputs, labels):
    mse = ((outputs - labels) ** 2).mean(axis=1)
    norm = (labels ** 2).mean(axis=1)
    nmse = mse / norm
    return nmse.mean()

def compute_mae(outputs, labels):
    mae = torch.abs(outputs - labels).mean()
    return mae


In [17]:
model.eval()  # Set the model to evaluation mode

total_nmse = 0.0
total_mae = 0.0
total_samples = 0

with torch.no_grad():
    for i, (images, _) in enumerate(testloader):
        images = images.view(images.shape[0], -1).to(device)
        labels = test_singular_values[i]  # Use precomputed singular values

        outputs = model(images)
        nmse = compute_nmse(outputs, labels)
        mae = compute_mae(outputs, labels)

        total_nmse += nmse.item() * images.size(0)
        total_mae += mae.item() * images.size(0)
        total_samples += images.size(0)

average_nmse = total_nmse / total_samples
average_mae = total_mae / total_samples

print(f'Average NMSE: {average_nmse:.4f}')
print(f'Average MAE: {average_mae:.4f}')


Average NMSE: 0.0862
Average MAE: 0.2049
